In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Problem 3: Visualizing Classification Regions

The moons dataset is a simple built-in dataset from the [scikit-learn library](https://scikit-learn.org/stable/)

In [ ]:
# run this cell to generate and visualize the dataset
from sklearn.datasets import make_moons
X,y = make_moons(n_samples=1000,noise=0.2)

plt.figure(figsize=(12,5))
plt.plot(X[y==0,0],X[y==0,1],'o',label = 'class '+str(0))
plt.plot(X[y==1,0],X[y==1,1],'o',label = 'class '+str(1))
plt.legend(fontsize=15)
plt.title('Moons dataset', fontsize=20)

## Part 1

Fit a **logistic regression** model to the data.

## Part 2

Your logistic regression model will split the feature space into regions called **classification regions**. 
All points in a classification region are assigned to the same class.
Run the following two cells to visualize the decision regions.
Notice that the boundary between regions is linear.

In [ ]:
def plot_classification_regions(X, y, theta, degree=0, bias=False):
    from matplotlib.colors import ListedColormap
    from itertools import chain
    from itertools import combinations_with_replacement as comb_w_r
    
    # create a 100x100 meshgrid
    m_plot = 100
    x1 = np.linspace(X[:,0].min()-0.5, X[:,0].max()+0.5, m_plot)
    x2 = np.linspace(X[:,1].min()-0.5,X[:,1].max()+0.5, m_plot)
    X1, X2 = np.meshgrid(x1, x2) 
    X_plot = np.c_[X1.ravel(), X2.ravel()]
    
    
    
    # polynomial features
    if degree==0:
        X_plot_poly = X_plot
    else:
        n = 2
        # number of polynomial features
        combinations  = chain.from_iterable(comb_w_r(range(n), i) for i in range(1, degree+1))
        n_poly = sum(1 for comb in combinations)
        # add polynomial features
        combinations  = chain.from_iterable(comb_w_r(range(n), i) for i in range(1, degree+1))
        X_plot_poly = np.ones((m_plot**2,n_poly))
        for col_idx,combination in enumerate(combinations):  
            for idx in combination:
                X_plot_poly[:,col_idx]= X_plot_poly[:,col_idx]*X_plot[:,idx]
    # add bias            
    if bias:
        X_plot_poly = np.c_[np.ones(m_plot**2),X_plot_poly]

    # evaluate the log regression model  at each point of the mesh grid    
    y_plot = sigmoid(X_plot_poly.dot(theta))
        
    # class prediction    
    y_plot[y_plot>=0.5]=1
    y_plot[y_plot<0.5]=0
    y_plot = y_plot.reshape(X1.shape)

    custom_cmap = ListedColormap(['#9898ff','#fafab0'])
    # contour map
    colors = ['blue','red','green']
    plt.figure(figsize=(12,5))
    plt.contourf(X1, X2, y_plot, alpha=0.3, cmap=custom_cmap)
    
    
    plt.plot(X[y==0,0],X[y==0,1],'o',label = 'class '+str(0))
    plt.plot(X[y==1,0],X[y==1,1],'o',label = 'class '+str(1))
    plt.legend(fontsize=15)
    plt.title('Moons dataset', fontsize=20)


In [ ]:
# run this cell to visualize the classification regions
plot_classification_regions(X, y, theta)

## Part 3

Add the polynomial features of degree 3 and the bias term to your logistic regression model.

In [ ]:
degree = 3

## Part 4

Run the following cell to visualize the new decision regions.

In [ ]:
plot_classification_regions(X, y, theta, degree=degree, bias=True)